In [10]:
import os
import pandas as pd

import logging
import time
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# NEEDS TO BE ABSOLUTE PATH
PATH_TO_DOWNLOADS = "/Users/mdong/dataScience/projects-ml/waste-management/web-scraping/calrecycle-data"
options = Options()
prefs = {'download.default_directory' : PATH_TO_DOWNLOADS}
options.add_experimental_option('prefs', prefs)
options.headless = False

how to iterate through each county? 

- each county doesn't map to URL like on waste characterization page
- use the n-th child selector - one for each county

### get it working for one county

In [11]:
DATA_URL = "https://www2.calrecycle.ca.gov/LGCentral/DisposalReporting/Origin/CountywideSummary"

driver = webdriver.Chrome(options=options)

driver.get(DATA_URL)

county_element = driver.find_element_by_css_selector('#CountyID > option:nth-child(5)')
county = county_element.text
county_element.click()

driver.find_element_by_css_selector('#SearchButton').click() # .send_keys("\n")

most_recent_waste_data = PATH_TO_DOWNLOADS + "/CountywideSummary.xlsx"
print("Downloading data for county: {}".format(county_element.text))

seconds_waited = 0

while not os.path.exists(most_recent_waste_data):
    time.sleep(1)
    seconds_waited += 1

print("Download took {} seconds...".format(seconds_waited))

driver.quit()

Download took 1 seconds...
